In [1]:
from data_utils import extract_data, load_data, preprocess_data

In [2]:
data = load_data("data")

In [3]:
X_train, X_test, y_train, y_test = preprocess_data(
    data,
    add_post_text=True,
    include_pooled_comments=False,
    add_bertscore=True,
    n_negative_samples_per_link=1
)

C:\Users\Raman\PycharmProjects\stackexchange-recsys\data_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_features['body'] = posts_features['body'].map(clean_html)


Calculating BERTScore (title):   0%|          | 0/160 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Calculating BERTScore (body):   0%|          | 0/160 [00:00<?, ?it/s]

Calculating BERTScore (title recall):   0%|          | 0/160 [00:00<?, ?it/s]

Calculating BERTScore (title_related recall):   0%|          | 0/160 [00:00<?, ?it/s]

In [4]:
# for col in X_train.columns:
#     print(X_train[col].)
X_train['body'].iloc[0]

'Our freezer was accidentally switched off approx a week ago and it was full of beef which had been frozen when very fresh will it be safe to refreeze\n'

In [5]:
X_train

,linktypeid,score,viewcount,answercount,commentcount,body,title,score_related,viewcount_related,answercount_related,...,vote_count,vote_count_related,avg_comment_score,comment_count,avg_comment_score_related,comment_count_related,bertscore_title_f1,bertscore_body_f1,bertscore_title_recall,bertscore_title_related_recall
3802,1,1,369,1,0,Our freezer was accidentally switched off appr...,Thawed meat can we refreeze,48,297710,10,...,2.0,57.0,NaN,NaN,1.000000,2.0,0.906816,0.835695,0.893248,0.884453
2500,1,14,4250,5,1,Chutney is foreign to my culture and the food ...,What is a Chutney?,102,22100,6,...,14.0,163.0,0.000000,1.0,0.270833,48.0,0.805306,0.795133,0.811515,0.780576
2822,0,1,954,1,9,When I try to make chicken soup I usually find...,Boiled chicken: coloured parts of the meat?,0,684,4,...,3.0,NaN,0.222222,9.0,NaN,NaN,0.853588,0.825696,0.844871,0.834686
4406,0,2,12570,2,1,Most store-bought stock/broth packaging contai...,Refrigerating Clam Juice,-1,91,1,...,2.0,1.0,0.000000,1.0,1.000000,1.0,0.866808,0.828586,0.818342,0.788917
1530,0,1,767,1,3,I bought a set of three Magnalite anodized fry...,Seasoning on Magnalite anodized aluminum fry p...,3,525,1,...,2.0,3.0,0.666667,3.0,0.000000,4.0,0.833207,0.834117,0.812912,0.862369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,0,5,456,2,0,My fiancée and I recently got a bunch of new (...,Using a large pot/pan on a smaller burner,1,63,1,...,5.0,1.0,NaN,NaN,1.000000,1.0,0.811381,0.825522,0.822001,0.784989
5755,0,1,440,2,3,I once heard that it is possible to make coffe...,coffee from seeds of dates,20,37717,1,...,1.0,21.0,1.000000,3.0,0.500000,2.0,0.810579,0.833758,0.819576,0.820038
896,0,6,130,3,0,I would like to purchase some very nice sea sa...,What is a good online resource for purchasing ...,1,89,1,...,6.0,1.0,NaN,NaN,0.000000,6.0,0.824541,0.819089,0.838555,0.812821
8134,0,2,1113,2,3,"Over the holidays, my mom tasked me to prepare...",How do I adjust cooking time when temperature ...,12,8386,3,...,3.0,15.0,1.333333,3.0,NaN,NaN,0.871597,0.787249,0.851840,0.836848


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6391 entries, 3802 to 7779
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   linktypeid                      6391 non-null   int64  
 1   score                           6391 non-null   object 
 2   viewcount                       6391 non-null   object 
 3   answercount                     6391 non-null   object 
 4   commentcount                    6391 non-null   object 
 5   body                            6391 non-null   object 
 6   title                           6391 non-null   object 
 7   score_related                   6391 non-null   object 
 8   viewcount_related               6391 non-null   object 
 9   answercount_related             6391 non-null   object 
 10  commentcount_related            6391 non-null   object 
 11  body_related                    6391 non-null   object 
 12  title_related                   6391

In [7]:
X_train["linktypeid"].value_counts()

linktypeid
0    3408
1    2983
Name: count, dtype: int64

In [8]:
X_test["linktypeid"].value_counts()

linktypeid
0    864
1    734
Name: count, dtype: int64

In [9]:
X_train.drop(columns=["linktypeid"], inplace=True)
X_test.drop(columns=["linktypeid"], inplace=True)

In [10]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

# Define categorical features indices if you have any
categorical_features_indices = []  # e.g., [0, 1, 2] if your first three columns are categorical

# Initialize CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,  # Number of boosting iterations
    learning_rate=0.1,  # Learning rate
    depth=6,  # Depth of the tree
    cat_features=categorical_features_indices,  # Specify categorical features
    # text_features=["body", "comments_text", "comments_text_related", "body_related"],
    text_features=["body", "body_related", "title", "title_related"],
    verbose=100  # Print training progress every 100 iterations
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6315133	test: 0.6326601	best: 0.6326601 (0)	total: 509ms	remaining: 8m 28s
100:	learn: 0.2642262	test: 0.3137477	best: 0.3137477 (100)	total: 26.7s	remaining: 3m 58s
200:	learn: 0.1968875	test: 0.2959296	best: 0.2959296 (200)	total: 52.4s	remaining: 3m 28s
300:	learn: 0.1597076	test: 0.2886522	best: 0.2886522 (300)	total: 1m 18s	remaining: 3m 2s
400:	learn: 0.1337232	test: 0.2860386	best: 0.2860386 (400)	total: 1m 44s	remaining: 2m 35s
500:	learn: 0.1127942	test: 0.2855983	best: 0.2852854 (461)	total: 2m 19s	remaining: 2m 18s
600:	learn: 0.0968826	test: 0.2854647	best: 0.2842441 (565)	total: 2m 44s	remaining: 1m 49s
700:	learn: 0.0831255	test: 0.2844954	best: 0.2841783 (663)	total: 3m 11s	remaining: 1m 21s
800:	learn: 0.0724122	test: 0.2849465	best: 0.2841783 (663)	total: 3m 36s	remaining: 53.9s
900:	learn: 0.0635072	test: 0.2859470	best: 0.2841783 (663)	total: 4m 2s	remaining: 26.7s
999:	learn: 0.0557661	test: 0.2875411	best: 0.2841783 (663)	total: 4m 28s	remaining: 0us

b

In [11]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,body,17.716821
1,body_related,17.473914
2,bertscore_title_related_recall,14.949340
3,bertscore_title_recall,9.968813
4,title_related,8.653108
5,vote_count_related,6.792407
6,title,6.151979
7,bertscore_body_f1,5.753149
8,score_related,4.258330
9,viewcount_related,2.385446
